In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import wordcloud


import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
# font_name 으로 변수 지정해주고, matpolotlib에서 그 폰트를 사용할 수 있게 설정해주기

plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

import xml.etree.ElementTree as et
import urllib.request as req
import requests # Error 발생 안함, Error를 건너 뜀

import json
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # pip install webdriver_manager 설치
import time

In [ ]:
# 사이트 원격 접속 (서울로 지정)
url = "https://tago.kr/charge/address.htm?&area=%EC%84%9C%EC%9A%B8&area2=all&pageNum=1"
driver = webdriver.Chrome("chromedriver")
driver.get(url)

In [ ]:
## 시군구 목록 가져오기
gu_list_0 = driver.find_element(By.ID, "area2")
gu_list = gu_list_0.find_elements(By.TAG_NAME, "option")

gu_names = [n.get_attribute("value") for n in gu_list]
gu_names.remove("all") # 처음에 필요없는 all삭제
gu_names

In [ ]:
## 구 선택하기
sel_gu = driver.find_element(By.NAME, "area2")
sel_gu.send_keys(gu_names[0])

In [ ]:
# 조회 버튼 누르기
driver.find_element(By.CLASS_NAME, "search-btn").click()

In [ ]:
# 엑셀 다운로드
driver.find_element(By.ID,"btndown").click()

In [ ]:
######### 지역별(구별) 전기차 충전소 엑셀로 저장

from tqdm.notebook import tqdm

for gu in tqdm(gu_names):
    sel_gu = driver.find_element(By.NAME, "area2")
    sel_gu.send_keys(gu)
    
    time.sleep(0.5)
    
    # 조회하기
    driver.find_element(By.CLASS_NAME, "search-btn").click()
    time.sleep(0.5)
    
    # 엑셀 저장
    driver.find_element(By.ID,"btndown").click()
    time.sleep(0.5)

driver.close()

In [ ]:
from glob import glob
import pandas as pd
import openpyxl as op
import xlwings as xw
import xlrd
import win32com
import win32com.client as win32
from openpyxl import load_workbook
from xlrd import open_workbook

In [ ]:
el_station_files = glob("data/el_station/xlsx/충전소*.*")
el_station_files

# station_raw = []
# for f in el_station_files:
#     file = pd.read_excel(f, header = 1)
#     station_raw.append(file)

In [ ]:
import win32com

import win32com.client as win32

fname = "C:\ww\Team_middle_project\data\el_station\충전소 리스트-20221022212546.xls"
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fname)

wb.SaveAs(fname+"x", FileFormat = 51)    #FileFormat = 51 is for .xlsx extension
wb.Close()                               #FileFormat = 56 is for .xls extension
excel.Application.Quit()

In [ ]:
df = pd.read_excel("C:\\ww\\Team_middle_project\\data\\el_station\\xlsx\\충전소 리스트-20221022212451.xlsx")
            
df

In [ ]:
# Data Frame에 넣기
from glob import glob
estation_files = glob("data/el_station/xlsx/충전소*.*")
estation_files

raw_file = []
for f in estation_files:
    rf = pd.read_excel(f, header=0) # 첨부터 시작
    raw_file.append(rf)
    
raw_file
# len(raw_file)

In [ ]:
# 데이터 프레임으로 합치기
estation_raw = pd.concat(raw_file)
estation_raw.head(50)

In [ ]:
estation_raw.info()

In [ ]:
# 데이터 정제
estation = pd.DataFrame({"충전소명":estation_raw["충전소명"],
                        "주소":estation_raw["소재지 도로명 주소"],                                           
                        "위도":estation_raw["위도"],
                        "경도":estation_raw["경도"],
                        })
estation.head(3)
## 인덱스 조정
estation.reset_index(inplace=True)
estation

In [ ]:
# 구별 나누기
estation["구"] = [n.split()[1] for n in estation["주소"]]
estation.head(3)


In [ ]:
## 지도에 표시하기
import requests

url = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}".format('세종로공영주차장')
header = {"Authorization":"KakaoAK 3589a39c9649be8b722eb90814d28726"}
requests.get(url, headers = header).json()

In [ ]:
#estation.info()
################### 시각화

In [ ]:
# 서울지역 전체 충전소

map = folium.Map(location=[37.497911, 127.027558],zoom_start=10)
  

for n in estation.index:
    lat = estation["위도"][n]
    lng = estation["경도"][n]
    folium.Marker(location =[lat,lng],
                  icon=folium.Icon(color='green',icon='heart'),
                  popup = estation["충전소명"][n]).add_to(map)
map

In [ ]:
# 구별로 찍기
estation[estation["구"] =="강남구"]